<a href="https://colab.research.google.com/github/yamatai-wizard/GCI_competition1/blob/main/Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. ライブラリ・データ読み込み

In [ ]:
!pip install optuna
!pip install -U xgboost


# Google ColabのGoogleドライブファイル保存用
from google.colab import files

# 基本的なライブラリのインポート
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import re

# グラフ描画用ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns

# 機械学習用ライブラリのインポート
from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import xgboost
import lightgbm
import optuna

# 警告メッセージを非表示にする
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 訓練データとテストデータを読み込み
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
all = pd.concat([df, df_test])
answer = pd.read_csv("answer.csv")
answer["Survived"]

,Survived
0,1
1,0
2,1
3,1
4,0
...,...
413,1
414,0
415,1
416,1


# 1. データの概観・分析・前処理

## 1.1 データの概観

In [ ]:
# print(df.shape)
# print(df_test.shape)

(891, 12)
(418, 11)


In [ ]:
# df.head(10)

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,1,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,1,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## 1.2 データの分析

In [ ]:
# df.isnull().sum()

,0
PassengerId,0
Perished,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [ ]:
# df_test.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [ ]:
# sns.boxplot(x="Pclass", y="Fare",data=df,showmeans=True)
# plt.show()

# round(df.groupby("Pclass")["Fare"].mean(),2)

In [ ]:
# df["Title"] = df["Name"].str.extract(r" ([A-Za-z]+)\.", expand = False)
# df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
# print(df.groupby('Title')['Age'].apply(lambda x: x.isnull().sum()))

# df_test["Title"] = df_test["Name"].str.extract(r" ([A-Za-z]+)\.", expand = False)
# df_test['Title'] = df_test['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
# print(df_test.groupby('Title')['Age'].apply(lambda x: x.isnull().sum()))

# sns.boxplot(x="Title", y="Age",data=df,showmeans=True)
# plt.xlabel("Title")
# plt.ylabel("Age")
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

# round(df.groupby("Title")["Age"].mean(),2)

In [ ]:
# df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
# df["IsAlone"] = (df["FamilySize"]==1).astype(int)

# # 生存率の計算
# survival_rate = df.groupby("IsAlone")["Perished"].mean()
# sample_size = df["IsAlone"].value_counts().sort_index()

# # サブプロット作成
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# # 生存率の円グラフ
# labels = ['With Family (0)', 'Alone (1)']
# colors = ['#ff9999', '#66b3ff']

# ax1.pie(survival_rate,
#         labels=labels,
#         autopct='%1.1f%%',
#         colors=colors,
#         startangle=90,
#         wedgeprops={'edgecolor': 'black'})
# ax1.set_title("Perished Rate by Group", fontsize=14, pad=20)

# # サンプルサイズの棒グラフ
# sns.barplot(x=sample_size.index,
#             y=sample_size.values,
#             palette="pastel",
#             edgecolor="black",
#             ax=ax2)

# ax2.set_xticks([0, 1])
# ax2.set_xticklabels(labels)
# ax2.set_title("Sample Size Comparison", fontsize=14, pad=20)
# ax2.set_xlabel("IsAlone Group", fontsize=12)
# ax2.set_ylabel("Number of Passengers", fontsize=12)

# # 数値ラベルの追加
# for p in ax2.patches:
#     ax2.annotate(f"{p.get_height():.0f}",
#                 (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='center',
#                 xytext=(0, 5),
#                 textcoords='offset points')

# plt.tight_layout()
# plt.show()

## 1.3 データの前処理

In [ ]:
# フォーク元https://www.kaggle.com/code/andreipaulavets/titanic-prediction-0-794-score
def data_cleaner(df):
    df = df.set_index("PassengerId")

    df["Title"] = df["Name"].str.extract(r" ([A-Za-z]+)\.", expand = False)
    df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})

    title_age_means = {
        'Mr': 	32.37,
        'Miss': 21.85,
        'Mrs': 35.79,
        'Master': 4.57,
        'Dr': 42.00
    }
    df['Age'].fillna(df['Title'].map(title_age_means), inplace=True)

    df.loc[~df['Title'].isin(['Mr', 'Miss', 'Mrs', 'Master',"Dr"]), 'Title'] = df['Sex'].map({'male': 'Mr', 'female': 'Mrs'})

    df["Embarked"].fillna(df["Embarked"].mode(), inplace=True)

    fare_means = {
        1: 84.15,
        2: 20.66,
        3: 13.68
    }
    df["Fare"].fillna(df["Pclass"].map(fare_means), inplace=True)

    df_sex = pd.get_dummies(df['Sex'], prefix='sex', drop_first=True, dtype=int)
    df_Pclass = pd.get_dummies(df['Pclass'], prefix='class', drop_first=True, dtype=int)
    df_Embarked = pd.get_dummies(df['Embarked'], prefix='Embarked', drop_first=True, dtype=int)
    df_Title = pd.get_dummies(df['Title'], prefix='Title', drop_first=False, dtype=int)

    df = pd.concat([df, df_sex, df_Pclass, df_Embarked, df_Title], axis=1)

    # df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    # df["IsAlone"] = (df["FamilySize"]==1).astype(int)

    # # 年齢は非線形的に影響すると予想
    # df['AgeBand'] = pd.cut(df['Age'], [0, 12, 20, 40, 60, np.inf], labels=[0, 1, 2, 3, 4]).astype(int)
    # # 値段の外れ値の影響を緩和
    # df['FareBand'] = pd.qcut(df['Fare'], 4, labels=[0, 1, 2, 3]).astype(int)

    df = df.drop(['Sex','Pclass','Name','Ticket','Embarked','Cabin','Title'], axis=1)

    return df

In [ ]:
df = data_cleaner(df)
df.head()

,Perished,Age,SibSp,Parch,Fare,sex_male,class_2,class_3,Embarked_Q,Embarked_S,Title_Dr,Title_Master,Title_Miss,Title_Mr,Title_Mrs
PassengerId,,,,,,,,,,,,,,,
1,1,22.0,1,0,7.2500,1,0,1,0,1,0,0,0,1,0
2,0,38.0,1,0,71.2833,0,0,0,0,0,0,0,0,0,1
3,0,26.0,0,0,7.9250,0,0,1,0,1,0,0,1,0,0
4,0,35.0,1,0,53.1000,0,0,0,0,1,0,0,0,0,1
5,1,35.0,0,0,8.0500,1,0,1,0,1,0,0,0,1,0


In [ ]:
df_test = data_cleaner(df_test)
df_test.head()

,Age,SibSp,Parch,Fare,sex_male,class_2,class_3,Embarked_Q,Embarked_S,Title_Dr,Title_Master,Title_Miss,Title_Mr,Title_Mrs
PassengerId,,,,,,,,,,,,,,
892,34.5,0,0,7.8292,1,0,1,1,0,0,0,0,1,0
893,47.0,1,0,7.0000,0,0,1,0,1,0,0,0,0,1
894,62.0,0,0,9.6875,1,1,0,1,0,0,0,0,1,0
895,27.0,0,0,8.6625,1,0,1,0,1,0,0,0,1,0
896,22.0,1,1,12.2875,0,0,1,0,1,0,0,0,0,1


# 2. ベースラインモデルの構築

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

X_test = df_test.iloc[:,:].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [118]:
def objective_rf(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 42
    }
    rfc = RandomForestClassifier(**param, n_jobs=-1)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(rfc, X_train, y_train, cv=cv, scoring="accuracy").mean()
    return score

# 最適化実行
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective_rf, n_trials=10)

# 最適パラメータの取得
best_params = study.best_params

# 最適パラメータでモデル構築
rfc = RandomForestClassifier(
    **best_params,
    n_jobs=-1,
    random_state=42
)

# 全データで学習
rfc.fit(X_train, y_train)

# 評価
print("RandomForest:\n")
print("Train Score: {}".format(round(rfc.score(X_train, y_train), 3)))
print("Test Score: {}".format(round(rfc.score(X_valid, y_valid), 3)))

rfc_pred = rfc.predict(X_test)
round(accuracy_score(answer["Survived"], rfc_pred),3)

[I 2025-05-23 03:05:24,955] A new study created in memory with name: no-name-4167a671-8093-4423-9238-90000f999634
[I 2025-05-23 03:05:28,243] Trial 0 finished with value: 0.824941935483871 and parameters: {'n_estimators': 144, 'max_depth': 15, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 0.824941935483871.
[I 2025-05-23 03:05:32,813] Trial 1 finished with value: 0.8136645161290323 and parameters: {'n_estimators': 227, 'max_depth': 3, 'min_samples_split': 20, 'min_samples_leaf': 9, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.824941935483871.
[I 2025-05-23 03:05:36,251] Trial 2 finished with value: 0.8345548387096773 and parameters: {'n_estimators': 158, 'max_depth': 6, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': True}. Best is trial 2 with value: 0.8345548387096773.
[I 2025-05-23 03:05:39,892] Trial 3 finished with value: 0.8281290322580646 and

RandomForest:

Train Score: 0.883
Test Score: 0.81


0.787

# 3. 特徴量エンジニアリング

# 4. 様々なモデルの構築・調整

In [119]:
# フォーク元https://cysec148.hatenablog.com/entry/2025/03/29/180502

def objective_xgb(trial):
    param = {
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "gamma": trial.suggest_float("gamma", 0, 1)
    }
    xgb = xgboost.XGBClassifier(**param, use_label_encoder=False, eval_metric="logloss")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(xgb, X_train, y_train, cv=cv, scoring="accuracy").mean()
    return score

# 最適化実行
study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective_xgb, n_trials=10)

# 最適パラメータ取得
best_params = study.best_params

# モデル構築
xgb = xgboost.XGBClassifier(
    **best_params,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42,
    early_stopping_rounds=50
)

# 全訓練データで学習（early stopping）
xgb.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=False
)


print("XGBoost \n")
print("Train Score: {}".format(round(xgb.score(X_train, y_train), 3)))
print(" Test Score: {}".format(round(xgb.score(X_valid, y_valid), 3)))

xgb_pred = xgb.predict(X_test)
round(accuracy_score(answer["Survived"], xgb_pred),3)

[I 2025-05-23 03:06:02,089] A new study created in memory with name: no-name-0f854d5b-0d33-4817-a479-78f3fc34eb13
[I 2025-05-23 03:06:03,789] Trial 0 finished with value: 0.8329161290322581 and parameters: {'learning_rate': 0.11861663446573512, 'max_depth': 10, 'gamma': 0.7319939418114051}. Best is trial 0 with value: 0.8329161290322581.
[I 2025-05-23 03:06:04,465] Trial 1 finished with value: 0.8265806451612903 and parameters: {'learning_rate': 0.18361096041714062, 'max_depth': 4, 'gamma': 0.15599452033620265}. Best is trial 0 with value: 0.8329161290322581.
[I 2025-05-23 03:06:05,239] Trial 2 finished with value: 0.8249290322580645 and parameters: {'learning_rate': 0.026844247528777843, 'max_depth': 9, 'gamma': 0.6011150117432088}. Best is trial 0 with value: 0.8329161290322581.
[I 2025-05-23 03:06:05,865] Trial 3 finished with value: 0.8233548387096775 and parameters: {'learning_rate': 0.21534104756085318, 'max_depth': 3, 'gamma': 0.9699098521619943}. Best is trial 0 with value: 0.8

XGBoost 

Train Score: 0.891
 Test Score: 0.799


0.766

In [120]:
# フォーク元https://cysec148.hatenablog.com/entry/2025/03/29/180502

def objective_lgb(trial):
    param = {
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 30),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        "verbosity": -1,
        "force_col_wise": True
    }
    lgb = lightgbm.LGBMClassifier(**param, objective="binary", metric="binary_logloss", random_state=42)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(lgb, X_train, y_train, cv=cv, scoring="accuracy").mean()
    return score

# 最適化実行
study = optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective_lgb, n_trials=10)

# 最適パラメータの取得
best_params = study.best_params

# 最適パラメータでモデル構築
lgb = lightgbm.LGBMClassifier(
    **best_params,
    objective="binary",
    metric="binary_logloss",
    random_state=42
)

# 全データで学習 (early stopping)
lgb.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="binary_logloss",
    callbacks=[lightgbm.early_stopping(stopping_rounds=50)],
)

# 評価
print("LightGBM:\n")
print("Train Score: {}".format(round(lgb.score(X_train, y_train), 3)))
print("Test Score: {}".format(round(lgb.score(X_valid, y_valid), 3)))

lgb_pred = lgb.predict(X_test)
round(accuracy_score(answer["Survived"], lgb_pred),3)

[I 2025-05-23 03:06:12,195] A new study created in memory with name: no-name-308746b4-7bb0-4b00-8b47-c58f7033d39e
[I 2025-05-23 03:06:13,033] Trial 0 finished with value: 0.8057806451612904 and parameters: {'learning_rate': 0.11861663446573512, 'num_leaves': 144, 'max_depth': 8, 'min_child_samples': 20, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 1, 'reg_alpha': 0.8661761457749352, 'reg_lambda': 0.6011150117432088}. Best is trial 0 with value: 0.8057806451612904.
[I 2025-05-23 03:06:13,892] Trial 1 finished with value: 0.8073548387096775 and parameters: {'learning_rate': 0.21534104756085318, 'num_leaves': 22, 'max_depth': 10, 'min_child_samples': 26, 'feature_fraction': 0.6061695553391381, 'bagging_fraction': 0.5909124836035503, 'bagging_freq': 2, 'reg_alpha': 0.3042422429595377, 'reg_lambda': 0.5247564316322378}. Best is trial 1 with value: 0.8073548387096775.
[I 2025-05-23 03:06:14,852] Trial 2 finished with value: 0.802541935483870

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.46332
LightGBM:

Train Score: 0.82
Test Score: 0.817


0.782

# 5. モデルのアンサンブリング

In [122]:
rfc_pred = rfc.predict_proba(X_test)
xgb_pred = xgb.predict_proba(X_test)
lgb_pred = lgb.predict_proba(X_test)

# 各モデルの検証スコアを取得
val_scores = {
    "rfc": rfc.score(X_valid, y_valid),
    "xgb": xgb.score(X_valid, y_valid),
    "lgb": lgb.score(X_valid, y_valid)
}

# 相対重みを計算
total = sum(val_scores.values())
weights = {k: v/total for k, v in val_scores.items()}

# 重み付き平均
pred_proba = (
    xgb_pred * weights["xgb"] +
    lgb_pred * weights["lgb"] +
    rfc_pred * weights["rfc"]
)


pred = pred_proba.argmax(axis=1)
round(accuracy_score(answer["Survived"], pred),3)

0.778

# 6. 予測の出力・提出

In [ ]:
# gender_submission.csvファイルを読み込む
submission = pd.read_csv("gender_submission.csv")

# 予測結果を"Perished"列に代入
submission["Perished"] = pred

# 加工したデータをsubmission.csvとして保存（インデックス列は出力しない）
submission.to_csv("submission.csv", index=False)

# Google Colab上でファイルをダウンロード
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>